In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import sentencepiece as spm
import matplotlib.pyplot as plt
import multiprocessing
import time
from gensim.models import Word2Vec
from torch.utils.data import Dataset, DataLoader

## Load data

In [3]:
train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')
validate = pd.read_parquet('validate.parquet')

### collect all texts to one dataset

In [ ]:

# with open('AllTexts.txt', 'w') as f:
#     pass  # This just creates the file, immediately closing it

# with open('AllTexts.txt', 'a') as f:  # Open file in append mode
#     for _, row in train.iterrows():
#         concatenated = '\n'.join(row['passages']['passage_text'])
#         concatenated = '\n'.join([concatenated, '\n'.join(row['answers'])])
#         concatenated = '\n'.join([concatenated, row['query']])
#         f.write(concatenated.lower() + '\n')
#     for _, row in test.iterrows():
#         concatenated = '\n'.join(row['passages']['passage_text'])
#         concatenated = '\n'.join([concatenated, '\n'.join(row['answers'])])
#         concatenated = '\n'.join([concatenated, row['query']])
#         f.write(concatenated.lower() + '\n')
#     for _, row in validate.iterrows():
#         concatenated = '\n'.join(row['passages']['passage_text'])
#         concatenated = '\n'.join([concatenated, '\n'.join(row['answers'])])
#         concatenated = '\n'.join([concatenated, row['query']])
#         f.write(concatenated.lower() + '\n')

### train tokanizer

In [ ]:
# spm.SentencePieceTrainer.train(
#     input = 'AllTexts.txt',
#     model_prefix='spm_AllTexts', 
#     vocab_size=30000,
# )

## Load Tokanizer

In [2]:
sp = spm.SentencePieceProcessor()
sp.load('spm_AllTexts.model')

True

In [3]:
def tokenize_file(file_path, sp_processor):
    tokenized_sentences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Assuming each line in the file is a separate sentence or paragraph
            # Tokenize the line and add the list of tokens to the tokenized_sentences list
            tokenized_sentences.append(sp_processor.encode_as_pieces(line.strip()))
    return tokenized_sentences

### Tokinize all the Dataset

In [ ]:
# tokinized_sentences = tokenize_file("AllTexts.txt", sp)

### Export tokens to JSON

In [ ]:
# import json
# with open("Tokens_AllText.json", 'w', encoding='utf-8') as file:
#     json.dump(tokinized_sentences, file, ensure_ascii=False, indent=4)

### Define W2V

In [4]:
vector_size = 128

In [5]:
w2v_model = Word2Vec(
    min_count  =20,
    window     =10,
    vector_size=vector_size,
    sample     =6e-5, 
    alpha      = 0.03, 
    min_alpha  = 0.0007, 
    negative   = 20,
    workers    = multiprocessing.cpu_count() - 1
)

In [ ]:
# print(len(tokinized_sentences))
# w2v_model.build_vocab(tokinized_sentences)
# w2v_model.save("word2vec.model")

In [ ]:
# with open("word2vec_vocab.txt", 'w') as vocab_file:
#     for word in w2v_model.wv.key_to_index.keys():
#         vocab_file.write(word + '\n')

In [ ]:
# w2v_model.train(tokinized_sentences, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)
# w2v_model.save("word2vec.model")

# Load Word2Vec Model

In [6]:
w2v_model = Word2Vec.load("word2vec.model")

In [7]:
similar_words = w2v_model.wv.most_similar('▁hacker', topn=4)
print(similar_words)
print(w2v_model.wv.most_similar(sp.encode_as_pieces('Hacker'.lower())))

[('▁cyber', 0.5855585336685181), ('▁hack', 0.5675662159919739), ('▁malicious', 0.5665103197097778), ('▁malware', 0.5581293106079102)]
[('▁cyber', 0.5855585336685181), ('▁hack', 0.5675662159919739), ('▁malicious', 0.5665103197097778), ('▁malware', 0.5581293106079102), ('▁scam', 0.5384910702705383), ('▁spyware', 0.5197509527206421), ('▁legitimate', 0.5031879544258118), ('▁adware', 0.4846465289592743), ('▁pretend', 0.469952255487442), ('▁insider', 0.46461066603660583)]


In [8]:
def to_w2v_embedding(sp, text):
    tokens = sp.encode_as_pieces(text.lower())

    embeddings = []
    for token in tokens:
        if (token in w2v_model.wv): 
            embeddings.append(w2v_model.wv[token])

    return embeddings

### Triples for training

In [11]:
def prepareTriplesTokens(dataframe):
    triples = []
    for index, row in dataframe.iterrows():
        available_indices = list(dataframe.index)
        available_indices.remove(index)
        
        for relevant in row['passages']['passage_text']:
            random_index = np.random.choice(available_indices)
            random_doc_index = np.random.choice(
                list(
                    range(
                        len(dataframe.iloc[random_index]['passages']['passage_text'])
                    )
                )
            )

            irrelevant = dataframe.iloc[random_index]['passages']['passage_text'][random_doc_index]

            triples.append([
                row['query'],
                relevant,
                irrelevant,
            ])

    return triples

# train_triplets = prepareTriplesTokens(train)
# test_triplets = prepareTriplesTokens(test)
# validate_triplets = prepareTriplesTokens(validate)

train_triplets = pd.read_parquet('train_triplets.parquet')
test_triplets = pd.read_parquet('test_triplets.parquet')
validate_triplets = pd.read_parquet('validate_triplets.parquet')

In [12]:
print(train_triplets[:2])

         query                                           relevant  \
0  what is rba  Since 2007, the RBA's outstanding reputation h...   
1  what is rba  The Reserve Bank of Australia (RBA) came into ...   

                                          irrelevant  
0  This report describes the typical weather at t...  
1  1. district, community the vicar of a small pa...  


In [20]:
# # Convert the list of triples to a DataFrame
# columns = ['query', 'relevant', 'irrelevant']
# train_triplets = pd.DataFrame(train_triplets, columns=columns)
# test_triplets = pd.DataFrame(test_triplets, columns=columns)
# validate_triplets = pd.DataFrame(validate_triplets, columns=columns)


In [62]:
# test_triplets_embeddings = pd.DataFrame()

# test_triplets_embeddings['query_embeddings'] = test_triplets['query'].apply(lambda x: to_w2v_embedding(sp, x))
# test_triplets_embeddings['relevant_embeddings'] = test_triplets['relevant'].apply(lambda x: to_w2v_embedding(sp, x))
# test_triplets_embeddings['irrelevant_embeddings'] = test_triplets['irrelevant'].apply(lambda x: to_w2v_embedding(sp, x))

In [63]:
# Calculate the memory usage of each column in bytes, then sum them up, and convert to megabytes
# total_memory_mb = test_triplets_embeddings.memory_usage(deep=True).sum() / (1024 ** 2)

# print(f'Total DataFrame size: {total_memory_mb:.2f} MB')

Total DataFrame size: 134.55 MB


In [64]:
# train_triplets_embeddings = pd.DataFrame()
# train_triplets_embeddings['query_embeddings'] = train_triplets['query'].apply(lambda x: to_w2v_embedding(sp, x))
# train_triplets_embeddings['relevant_embeddings'] = train_triplets['relevant'].apply(lambda x: to_w2v_embedding(sp, x))
# train_triplets_embeddings['irrelevant_embeddings'] = train_triplets['irrelevant'].apply(lambda x: to_w2v_embedding(sp, x))

In [65]:
# Calculate the memory usage of each column in bytes, then sum them up, and convert to megabytes
# total_memory_mb = train_triplets_embeddings.memory_usage(deep=True).sum() / (1024 ** 2)

# print(f'Total DataFrame size: {total_memory_mb:.2f} MB')

Total DataFrame size: 1150.24 MB


In [66]:
# print(test_triplets_embeddings['query_embeddings'].head(1))

# train_triplets_embeddings.to_parquet('train_triplets_with_embedings.parquet', engine='pyarrow') 
# test_triplets_embeddings.to_parquet('test_triplets_with_embedings.parquet', engine='pyarrow') 
# validate_triplets.to_parquet('validate_triplets_with_embedings.parquet', engine='pyarrow') 

0    [[-0.38117662, -1.751613, 0.118526, -4.4650145...
Name: query_embeddings, dtype: object


# Dataset

In [175]:
class QueryDocsDataset(Dataset):
    def __init__(self, sp, queries, relevant_docs, irrelevant_docs):
        self.queries = queries
        self.relevant_docs = relevant_docs
        self.irrelevant_docs = irrelevant_docs

    def __len__(self):
        return len(self.queries)

    def __getitem__(self, idx):
        return {
            'query': torch.tensor(to_w2v_embedding(sp, self.queries[idx].lower()), dtype=torch.float),
            'relevant': torch.tensor(to_w2v_embedding(sp, self.relevant_docs[idx].lower()), dtype=torch.float),
            'irrelevant': torch.tensor(to_w2v_embedding(sp, self.irrelevant_docs[idx].lower()), dtype=torch.float),
        }

### Custom DataLoader Class

In [87]:
# class CustomDataLoader:
#     def __init__(self, dataset, batch_size, shuffle=False):
#         self.dataset = dataset
#         self.batch_size = batch_size
#         self.shuffle = shuffle

#     def __iter__(self):
#         self.idx = 0
#         if self.shuffle:
#             np.random.shuffle(self.dataset)
#         return self

#     def __next__(self):
#         if self.idx < len(self.dataset):
#             batch = self.dataset[self.idx:self.idx + self.batch_size]
#             self.idx += self.batch_size
#             return batch
#         else:
#             raise StopIteration

#     def __len__(self):
#         return len(self.dataset) // self.batch_size
    

### Fill Datasets

In [177]:
TrainingDataset = QueryDocsDataset(sp, train_triplets['query'], train_triplets['relevant'], train_triplets['irrelevant'])
TestingDataset = QueryDocsDataset(sp, test_triplets['query'], test_triplets['relevant'], test_triplets['irrelevant'])
ValidationDataset = QueryDocsDataset(sp, validate_triplets['query'], validate_triplets['relevant'], validate_triplets['irrelevant'])

# TrainingDataset = QueryDocsDataset(sp, train_triplets_embeddings['query_embeddings'], train_triplets_embeddings['relevant_embeddings'], train_triplets_embeddings['irrelevant_embeddings'])
# TestingDataset = QueryDocsDataset(sp, test_triplets_embeddings['query_embeddings'], test_triplets_embeddings['relevant_embeddings'], test_triplets_embeddings['irrelevant_embeddings'])
# ValidationDataset = QueryDocsDataset(sp, validate_triplets_embeddings['query'], validate_triplets_embeddings['relevant'], validate_triplets_embeddings['irrelevant'])

# Models

In [251]:
class QueryRNNCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(QueryRNNCell, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.weight_ih = nn.Parameter(torch.randn(input_size, hidden_size))  # Input to hidden weights
        self.weight_hh = nn.Parameter(torch.randn(hidden_size, hidden_size))  # Hidden to hidden weights
        
        self.bias_hh = nn.Parameter(torch.randn(hidden_size))  # Bias

    def forward(self, input, hidden):
        return torch.tanh(
            torch.mm(input, self.weight_ih) + torch.mm(hidden, self.weight_hh) + self.bias_hh
        )
    
class QueryRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(QueryRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_cell = QueryRNNCell(input_size, hidden_size)

    def forward(self, input):
        # Assuming input is of shape (batch, seq_len, input_size)
        batch_size, seq_len, _ = input.shape
        hidden = torch.zeros(batch_size, self.hidden_size)  # Initial hidden state
        for i in range(seq_len):
            
            mask = torch.any(input[:, i, :] != 0, dim=1).float().unsqueeze(1)  # Shape: (batch_size, 1)
            current_input = input[:, i, :]  
            
            current_hidden = self.rnn_cell(current_input, hidden)
            
            # Apply mask: Only update hidden state for non-padded inputs
            hidden = mask * current_hidden + (1 - mask) * hidden
            
        return hidden

# Two Towers

In [252]:
class TwoTowerModel(nn.Module):
    def __init__(self, embedding_size, hidden_size):
        super(TwoTowerModel, self).__init__()
        self.queryEncoder = QueryRNN(embedding_size, hidden_size)
        self.docEncoder = QueryRNN(embedding_size, hidden_size)

    def forward(self, query, relevant, irrelevant):
        query_embedding = self.queryEncoder(query)
        relevant_embedding = self.docEncoder(relevant)
        irrelevant_embedding = self.docEncoder(irrelevant)
        return query_embedding, relevant_embedding, irrelevant_embedding

## Lose Function

In [253]:
import torch.nn.functional as F

def triplet_loss_function_cosine(query, relevant_doc, irrelevant_doc, margin):
    # Compute cosine similarity (the output ranges from -1 to 1)
    relevant_similarity = F.cosine_similarity(query, relevant_doc)
    irrelevant_similarity = F.cosine_similarity(query, irrelevant_doc)
    
    # Convert similarities to distances (ranges from 0 to 2)
    relevant_distance = 1 - relevant_similarity
    irrelevant_distance = 1 - irrelevant_similarity
    
    # Compute the triplet loss
    triplet_loss = torch.clamp(margin + relevant_distance - irrelevant_distance, min=0)
    return triplet_loss.mean()

### Padding Function

In [254]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Extract lists of tensors for 'query', 'relevant', and 'irrelevant' from the batch
    query_tensors = [item['query'] for item in batch]
    relevant_tensors = [item['relevant'] for item in batch]
    irrelevant_tensors = [item['irrelevant'] for item in batch]
    
    # Pad sequences within each list to the same length
    query_padded = pad_sequence(query_tensors, batch_first=True, padding_value=0)
    relevant_padded = pad_sequence(relevant_tensors, batch_first=True, padding_value=0)
    irrelevant_padded = pad_sequence(irrelevant_tensors, batch_first=True, padding_value=0)
    
    # Return a dictionary with padded sequences
    return {
        'query': query_padded,
        'relevant': relevant_padded,
        'irrelevant': irrelevant_padded
    }

## padding test

In [255]:
# Convert the numpy array to a PyTorch tensor
tensors =[ 
    torch.tensor([
        [51, 92, 14, 71],
        [60, 20, 82, 86],
        [74, 74, 87, 99],
        [23,  2, 21, 52],
        [ 1, 87, 29, 37],
    ]),
    torch.tensor([
        [51, 92, 14, 71],
        [60, 20, 82, 86],
        [74, 74, 87, 99],
        [23,  2, 21, 52],
        [ 1, 87, 29, 37],
        [60, 20, 82, 86],
    ]),
]
dump = pad_sequence(tensors, batch_first=True)

print(dump)

tensor([[[51, 92, 14, 71],
         [60, 20, 82, 86],
         [74, 74, 87, 99],
         [23,  2, 21, 52],
         [ 1, 87, 29, 37],
         [ 0,  0,  0,  0]],

        [[51, 92, 14, 71],
         [60, 20, 82, 86],
         [74, 74, 87, 99],
         [23,  2, 21, 52],
         [ 1, 87, 29, 37],
         [60, 20, 82, 86]]])


# Initialize Model and Dataloader

In [269]:
embedding_size = 128
hidden_size = 16
margin = 1.0
batch_size = 200
num_epochs = 1

# TrainingDataloader = DataLoader(TrainingDataset, batch_size, shuffle=False)
# TestingDataloader = DataLoader(TestingDataset, batch_size, shuffle=False)

TrainingDataloader = DataLoader(TrainingDataset, batch_size, shuffle=False, collate_fn=collate_fn)
TestingDataloader = DataLoader(TestingDataset, batch_size, shuffle=False, collate_fn=collate_fn)
ValidatingDataloader = DataLoader(ValidationDataset, batch_size, shuffle=False, collate_fn=collate_fn)

# Instantiate the model
Towers = TwoTowerModel(embedding_size, hidden_size)
optimizer = torch.optim.Adam(Towers.parameters(), lr=0.001)


## Recheck the padding results

In [212]:
for batch in TestingDataloader:
    print(batch['query'].shape)        # Shape: (batch_size, max_seq_length_query, feature_dim)
    print(batch['relevant'].shape)     # Shape: (batch_size, max_seq_length_relevant, feature_dim)
    print(batch['irrelevant'].shape) 

    for i in range(batch['query'].size(0)):
        print(f"Element {i+1}, last row of 7:\n{batch['query'][i, -1, :].numpy()}\n")

    break

torch.Size([20, 7, 128])
torch.Size([20, 134, 128])
torch.Size([20, 154, 128])
Element 1, last row of 7:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]

Element 2, last row of 7:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]

Element 3, last row of 7:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [270]:


for epoch in range(num_epochs):
    Towers.train() 
    train_loss = 0.0

    i = 0
    for batch in TestingDataloader:
        i +=1
        optimizer.zero_grad()

        # Forward pass through the model to get embeddings
        query_embeddings, relevant_doc_embeddings, irrelevant_doc_embeddings = Towers(
            batch['query'], 
            batch['relevant'], 
            batch['irrelevant']
        )

        # Compute the loss
        loss = triplet_loss_function_cosine(query_embeddings, relevant_doc_embeddings, irrelevant_doc_embeddings, margin)

        # print(query_embeddings)
        # print(relevant_doc_embeddings)
        # print(irrelevant_doc_embeddings)
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Batch: {i}, Train Loss: {loss.item():.4f}')
        
        # break
    
    # # Testing phase
    # Towers.eval()  # Set model to evaluation mode
    # test_loss = 0.0
    # with torch.no_grad():  # No need to track gradients for testing
    #     for batch in TestingDataloader:
    #         query_embeddings, relevant_doc_embeddings, irrelevant_doc_embeddings = Towers(
    #             batch['query'], batch['relevant_doc'], batch['irrelevant_doc']
    #         )
            
    #         loss = triplet_loss_function_cosine(query_embeddings, relevant_doc_embeddings, irrelevant_doc_embeddings, margin)
    #         test_loss += loss.item()
            
    # avg_train_loss = train_loss / len(TestingDataloader)
    # avg_test_loss = test_loss / len(TestingDataloader)
    
    # print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}')

Epoch [1/1], Batch: 1, Train Loss: 1.0460
Epoch [1/1], Batch: 2, Train Loss: 0.9949
Epoch [1/1], Batch: 3, Train Loss: 1.0174
Epoch [1/1], Batch: 4, Train Loss: 1.0061
Epoch [1/1], Batch: 5, Train Loss: 1.0147
Epoch [1/1], Batch: 6, Train Loss: 1.0037
Epoch [1/1], Batch: 7, Train Loss: 1.0045
Epoch [1/1], Batch: 8, Train Loss: 1.0030
Epoch [1/1], Batch: 9, Train Loss: 0.9893
Epoch [1/1], Batch: 10, Train Loss: 1.0435
Epoch [1/1], Batch: 11, Train Loss: 0.9585
Epoch [1/1], Batch: 12, Train Loss: 0.9795
Epoch [1/1], Batch: 13, Train Loss: 1.0259
Epoch [1/1], Batch: 14, Train Loss: 1.0026
Epoch [1/1], Batch: 15, Train Loss: 1.0158
Epoch [1/1], Batch: 16, Train Loss: 1.0381
Epoch [1/1], Batch: 17, Train Loss: 0.9921
Epoch [1/1], Batch: 18, Train Loss: 0.9759
Epoch [1/1], Batch: 19, Train Loss: 1.0445
Epoch [1/1], Batch: 20, Train Loss: 0.9955
Epoch [1/1], Batch: 21, Train Loss: 0.9629
Epoch [1/1], Batch: 22, Train Loss: 0.9904
Epoch [1/1], Batch: 23, Train Loss: 0.9831
Epoch [1/1], Batch: 